# Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType , IntegerType
from pyspark.sql.functions import trim, col

# Read Bronze table

In [0]:
# I am reading from the source which is the Bronze
df=spark.table("workspace.bronze.erp_loc_a101")

In [0]:
df.display()

# Silver Transformations

In [0]:
df.dtypes


##  Trimming

In [0]:
for field in df.schema.fields: 
  if isinstance(field.dataType, StringType):
    df = df.withColumn(field.name, trim(col(field.name)))
df.display()

In [0]:
df.groupBy("CNTRY").count().display()

In [0]:
# check for NULLs
null_stats = (
    df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in df.columns
    ])
)

display(null_stats)


## Country column normalization

In [0]:


df=df.withColumn("CNTRY",
                 F.when(col("CNTRY").isin("US", "USA"), "United States")
                 .when(col("CNTRY")=="DE","Germany")
                 .when((col("CNTRY").isNull()) | (col("CNTRY")==""),"Unknown")
                 .otherwise(col("CNTRY"))
                 )




In [0]:
df.groupBy("CNTRY").count().display()

## Customer ID cleanup

In [0]:
df=df.withColumn("CID",F.regexp_replace(col("CID"),"-", ""))

In [0]:
df.display()

## Renaming The Columns

In [0]:

RENAME_MAP = {
    "CID": "customer_number",
    "CNTRY": "country"
    
}

for old,new in RENAME_MAP.items():
    df = df.withColumnRenamed(old, new)
df.display()

# Sanity checks

In [0]:
null_stats = (
    df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in df.columns
    ])
)

display(null_stats)

In [0]:
#sanity check
df.limit(10).display()

# Write Into Silver Table
# 

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")

In [0]:
%sql
SELECT * FROM silver.erp_customer_location
LIMIT 5